In [ ]:
# Get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# This file contains all the main external libs we'll use
from fastai.imports import *

In [ ]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [ ]:
PATH = "../../data/food-101/"

In [ ]:
sz=224

In [ ]:
# util to plot imgs
def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

In [ ]:
# plot a random img from each class
folders = !ls {PATH}valid
imgs = []
for folder in folders:
    files = !ls {PATH}valid/{folder} | head
    random_choice = random.choice(files)
    img = open_image(os.path.join(PATH, 'valid', folder, random_choice))
    imgs.append(img)
    
plots(imgs[:100], figsize=(15, 50), rows = 10)

In [ ]:
# If you need to reset your precomputed activations
!rm -rf {PATH}tmp

In [ ]:
arch=resnext101
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz), test_name='test')
learn = ConvLearner.pretrained(arch, data, precompute=True, xtra_fc=[512, 512], ps=0.5)
# Uncomment to load trained model
learn.load('better_model')

In [ ]:
learn.fit(0.01, 30)
learn.fit(0.001, 30)
learn.fit(0.0001, 50)

# unfreeze frozen layers and train again
learn.unfreeze()
# learning rate dacays with deepness of layers
lr=np.array([1e-5,1e-4,1e-3])


learn.fit(lr, 20, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save('better_model')

In [ ]:
# print data classes
print(data.classes)

In [ ]:
# Prediction for validation set. Predictions are in log scale
log_preds = learn.predict()
log_preds.shape

In [ ]:
preds = np.argmax(log_preds, axis=1)  # from log probabilities to 0 or 1
probs = np.exp(log_preds)
print(preds)

In [ ]:
def topK(k):
    corrects = 0
    per_class = np.zeros(101)
    for i in range(len(probs)):
        c = int(i/100)
        if np.argmax(probs[i]) == c:
            per_class[c] += 1
        max_indexes = probs[i].argsort()[-k:][::-1]
        if c in max_indexes:
            corrects += 1
    print(corrects/10100)
    return per_class
 
per_class = topK(5)

In [ ]:
# top classes:
for i in range(len(per_class)):
    if per_class[i] > 80:
        print(per_class[i], data.classes[i])

In [ ]:
# worst classes:
for i in range(len(per_class)):
    if per_class[i] < 50:
        print(per_class[i], data.classes[i])

In [ ]:
# Data augmentation
tfms = tfms_from_model(resnext101, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
def get_augs():
    data = ImageClassifierData.from_paths(PATH, bs=2, tfms=tfms, num_workers=1)
    x,_ = next(iter(data.aug_dl))
    return data.trn_ds.denorm(x)[1]

In [ ]:
ims = np.stack([get_augs() for i in range(6)])

In [ ]:
plots(ims, rows=2)

In [ ]:
# Test time data augmentation
log_preds,y = learn.TTA()
preds = np.mean(np.exp(log_preds),0)

In [ ]:
accuracy(preds, y)